# 1. import

In [35]:
# 필요한 모델들을 import 합니다.

# 기본
import os
import json
import copy
from tqdm import tqdm

# 이미지 처리
import random
random.seed(42)

# torch Data
from torch.utils.data import DataLoader

# model 
import torch
import torch.nn as nn
import torch.optim as optim

#dataloader
from tools.SignalDataSet import SignalDataSet

#model
from tools.model import CNN_custom

# 2. device 설정

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# 3. 데이터

In [3]:
with open('config/custom.json', 'r') as file:
    data_dict = json.load(file)

train_dataset = SignalDataSet(data_dict['train'],data_dict['class'])
val_dataset = SignalDataSet(data_dict['val'],data_dict['class'])


# 4. model설정

In [33]:
# 모델 생성
model = CNN_custom(len(data_dict['class'])).double().to(DEVICE)

#파라미터 설정
EPOCHS = 10
BATCH_SIZE = 4
lr = 1e-5

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

# dataloader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 5. Train

In [36]:
for i in range(EPOCHS):
    best_model = None
    best_acc = 0

    # train
    print(f"Epochs {i+1}")
    train_loss_sum = 0
    for [image,label] in train_loader:
        x = image.double().to(DEVICE)
        y = label.to(DEVICE)

        output = model.forward(x)
        _,output_max = torch.max(output,1)

        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()
        train_loss_sum += loss.cpu().detach().numpy()
    print(f"train loss = {loss}")

    #val
    val_loss_sum = 0
    with torch.no_grad():
        total = 0
        correct = 0

        for [image,label] in val_loader:
            x = image.to(DEVICE)
            y = label.to(DEVICE)

            output = model.forward(x)
            _,output_max = torch.max(output,1)

            loss = loss_func(output,y)
            val_loss_sum += loss.cpu().detach().numpy()

            total += label.size(0)
            correct += (output_max == torch.argmax(y,1)).sum().cpu().detach().numpy()
            acc = correct / total

            if acc > best_acc:
                best_acc = acc
                best_model = copy.deepcopy(model)

    print(f"val loss = {loss}")
    print(f"val acc = {acc}")
    print("---------------------------------")


Epochs 1
train loss = 0.22771475670048646
val loss = 0.4083011002464426
val acc = 0.7739130434782608
---------------------------------
Epochs 2
train loss = 0.08525554235741015
val loss = 0.23763798973127642
val acc = 0.8782608695652174
---------------------------------
Epochs 3
train loss = 0.0012384736540945254
val loss = 0.00035289041770250737
val acc = 0.8173913043478261
---------------------------------
Epochs 4
train loss = 0.0005149753519714511
val loss = 0.00021492232836216546
val acc = 0.8521739130434782
---------------------------------
Epochs 5
train loss = 1.2953668149384886e-06
val loss = 0.09731227610163125
val acc = 0.9304347826086956
---------------------------------
Epochs 6
train loss = 1.0217121807904326
val loss = 1.2446152007007538e-07
val acc = 0.8782608695652174
---------------------------------
Epochs 7
train loss = 1.7662678785115518e-08
val loss = 9.570304115812875e-06
val acc = 0.9478260869565217
---------------------------------
Epochs 8
train loss = 0.00275

# 6. test & save

In [40]:
test_dataset = SignalDataSet(data_dict['test'],data_dict['class'])
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

best_model.eval()

count = 0
correct = 0

for [image,label] in test_loader:
    count+=1
    x = image.to(DEVICE)
    predict = best_model(x).argmax()
    gt = label.to(DEVICE).argmax()

    if predict == gt:
        correct+=1

print(f"acc = {correct/count*100}%")

folder_path = "result/" + data_dict['project_name']
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
torch.save(best_model.state_dict(),os.path.join(folder_path,'best.pth'))
    

acc = 96.12068965517241%
